In [1]:
import os

from IPython import display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_hub as hub

yamnet_model_handle = 'https://tfhub.dev/google/yamnet/1'
yamnet_model = hub.load(yamnet_model_handle)

2023-02-20 21:36:32.825025: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-20 21:36:35.224313: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# scan data directories
import glob

nature_file_list = []
nature_file_list += glob.glob('../data/interim/AmbisonicSoundLibrary/nature/*')
nature_file_list += glob.glob('../data/interim/GoogleAudioSet/Outside, rural or natural/*')
nature_file_list += glob.glob('../data/interim/youtube/NatureSoundscapes/*')
nature_file_list += glob.glob('../data/interim/youtube/NomadicAmbience_nature/*')
nature_file_list += glob.glob('../data/interim/S2L_LULC/non_urban/*')
nature_file_list += glob.glob('../data/interim/S2L_LULC/urban_0_25/*')

city_file_list = []
city_file_list += glob.glob('../data/interim/GoogleAudioSet/Outside, urban or manmade/*')
city_file_list += glob.glob('../data/interim/youtube/NomadicAmbience_city/*')
city_file_list += glob.glob('../data/interim/SONYC/**/*.pkl')
city_file_list += glob.glob('../data/interim/S2L_LULC/urban_26_100/*')

nature_source_list = ['nature_'+i.rsplit('/', 3)[1]+'/'+i.rsplit('/', 3)[2] for i in nature_file_list]
city_source_list = ['city_'+i.rsplit('/', -1)[3] for i in city_file_list]

In [3]:
nature_df = pd.DataFrame({'file': nature_file_list, 'source': nature_source_list, 'category': 0})
city_df = pd.DataFrame({'file': city_file_list, 'source': city_source_list, 'category': 1})
df_all = pd.concat([nature_df, city_df], ignore_index=True)
df_all

,file,source,category
0,../data/interim/AmbisonicSoundLibrary/nature/W...,nature_AmbisonicSoundLibrary/nature,0
1,../data/interim/AmbisonicSoundLibrary/nature/R...,nature_AmbisonicSoundLibrary/nature,0
2,../data/interim/AmbisonicSoundLibrary/nature/A...,nature_AmbisonicSoundLibrary/nature,0
3,../data/interim/AmbisonicSoundLibrary/nature/W...,nature_AmbisonicSoundLibrary/nature,0
4,../data/interim/AmbisonicSoundLibrary/nature/L...,nature_AmbisonicSoundLibrary/nature,0
...,...,...,...
1919,../data/interim/S2L_LULC/urban_26_100/s2lam111...,city_S2L_LULC,1
1920,../data/interim/S2L_LULC/urban_26_100/s2lam006...,city_S2L_LULC,1
1921,../data/interim/S2L_LULC/urban_26_100/s2lam083...,city_S2L_LULC,1
1922,../data/interim/S2L_LULC/urban_26_100/s2lam052...,city_S2L_LULC,1


In [46]:
# randomly sampled 80% as training, 10% as testing, and 10% and validation
test_df = df_all.groupby('source', group_keys=False).apply(lambda x: x.sample(frac=0.2, random_state=23))
val_df = test_df.groupby('source', group_keys=False).apply(lambda x: x.sample(frac=0.5, random_state=23))
# drop the ones got splitted out
train_df = df_all.drop(index = test_df.index)
test_df = test_df.drop(index = val_df.index)

test_df.reset_index(inplace=True, drop=True)
train_df.reset_index(inplace=True, drop=True)
val_df.reset_index(inplace=True, drop=True)

In [47]:
from sklearn.model_selection import StratifiedKFold

# Split the data into folds using StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=23)
for fold, (train_idx, val_idx) in enumerate(skf.split(train_df, train_df['source'])):
    # Assign the fold number to each row in the DataFrame
    train_df.loc[val_idx, 'fold'] = fold

In [48]:
train_df

,file,source,category,fold
0,../data/interim/AmbisonicSoundLibrary/nature/W...,nature_AmbisonicSoundLibrary/nature,0,1.0
1,../data/interim/AmbisonicSoundLibrary/nature/R...,nature_AmbisonicSoundLibrary/nature,0,1.0
2,../data/interim/AmbisonicSoundLibrary/nature/A...,nature_AmbisonicSoundLibrary/nature,0,4.0
3,../data/interim/AmbisonicSoundLibrary/nature/W...,nature_AmbisonicSoundLibrary/nature,0,3.0
4,../data/interim/AmbisonicSoundLibrary/nature/L...,nature_AmbisonicSoundLibrary/nature,0,3.0
...,...,...,...,...
1533,../data/interim/S2L_LULC/urban_26_100/s2lam038...,city_S2L_LULC,1,3.0
1534,../data/interim/S2L_LULC/urban_26_100/s2lam091...,city_S2L_LULC,1,1.0
1535,../data/interim/S2L_LULC/urban_26_100/s2lam111...,city_S2L_LULC,1,4.0
1536,../data/interim/S2L_LULC/urban_26_100/s2lam052...,city_S2L_LULC,1,2.0


In [5]:
test_df

,file,source,category
1058,"../data/interim/GoogleAudioSet/Outside, urban ...",city_GoogleAudioSet,1
1054,"../data/interim/GoogleAudioSet/Outside, urban ...",city_GoogleAudioSet,1
1099,"../data/interim/GoogleAudioSet/Outside, urban ...",city_GoogleAudioSet,1
1083,"../data/interim/GoogleAudioSet/Outside, urban ...",city_GoogleAudioSet,1
1009,"../data/interim/GoogleAudioSet/Outside, urban ...",city_GoogleAudioSet,1
...,...,...,...
325,../data/interim/youtube/NatureSoundscapes/Peh4...,nature_youtube/NatureSoundscapes,0
401,../data/interim/youtube/NatureSoundscapes/aEH1...,nature_youtube/NatureSoundscapes,0
474,../data/interim/youtube/NomadicAmbience_nature...,nature_youtube/NomadicAmbience_nature,0
471,../data/interim/youtube/NomadicAmbience_nature...,nature_youtube/NomadicAmbience_nature,0


In [6]:
# sanity check
test_df.groupby('source').count()

,file,category
source,,
city_GoogleAudioSet,17,17
city_S2L_LULC,24,24
city_SONYC,51,51
city_youtube,5,5
nature_AmbisonicSoundLibrary/nature,6,6
"nature_GoogleAudioSet/Outside, rural or natural",16,16
nature_S2L_LULC/non_urban,24,24
nature_S2L_LULC/urban_0_25,24,24
nature_youtube/NatureSoundscapes,23,23


In [7]:
val_df.groupby('source').count()

,file,category
source,,
city_GoogleAudioSet,17,17
city_S2L_LULC,24,24
city_SONYC,50,50
city_youtube,5,5
nature_AmbisonicSoundLibrary/nature,6,6
"nature_GoogleAudioSet/Outside, rural or natural",16,16
nature_S2L_LULC/non_urban,24,24
nature_S2L_LULC/urban_0_25,24,24
nature_youtube/NatureSoundscapes,23,23


In [8]:
train_df.groupby('source').count()

,file,category
source,,
city_GoogleAudioSet,136,136
city_S2L_LULC,192,192
city_SONYC,402,402
city_youtube,39,39
nature_AmbisonicSoundLibrary/nature,46,46
"nature_GoogleAudioSet/Outside, rural or natural",128,128
nature_S2L_LULC/non_urban,192,192
nature_S2L_LULC/urban_0_25,192,192
nature_youtube/NatureSoundscapes,184,184


In [9]:
train_df

,file,source,category
0,../data/interim/AmbisonicSoundLibrary/nature/W...,nature_AmbisonicSoundLibrary/nature,0
1,../data/interim/AmbisonicSoundLibrary/nature/R...,nature_AmbisonicSoundLibrary/nature,0
2,../data/interim/AmbisonicSoundLibrary/nature/A...,nature_AmbisonicSoundLibrary/nature,0
3,../data/interim/AmbisonicSoundLibrary/nature/W...,nature_AmbisonicSoundLibrary/nature,0
4,../data/interim/AmbisonicSoundLibrary/nature/L...,nature_AmbisonicSoundLibrary/nature,0
...,...,...,...
1917,../data/interim/S2L_LULC/urban_26_100/s2lam038...,city_S2L_LULC,1
1918,../data/interim/S2L_LULC/urban_26_100/s2lam091...,city_S2L_LULC,1
1919,../data/interim/S2L_LULC/urban_26_100/s2lam111...,city_S2L_LULC,1
1922,../data/interim/S2L_LULC/urban_26_100/s2lam052...,city_S2L_LULC,1


# Customize model

In [10]:
my_classes = ['city', 'nature']
my_model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(1024), dtype=tf.float32,
                          name='input_embedding'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(len(my_classes))
], name='my_model')

my_model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               524800    
                                                                 
 dense_1 (Dense)             (None, 2)                 1026      
                                                                 
Total params: 525,826
Trainable params: 525,826
Non-trainable params: 0
_________________________________________________________________


In [11]:
my_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                 optimizer="adam",
                 metrics=['accuracy'])

callback = tf.keras.callbacks.EarlyStopping(monitor='loss',
                                            patience=3,
                                            restore_best_weights=True)

In [12]:
type(my_model)

keras.engine.sequential.Sequential

In [13]:
def load_wav_pkl(filename):
    import pickle
    # open a file, where you stored the pickled data
    file = open(filename, 'rb')

    # dump information to that file
    output = pickle.load(file)
    wav = output['y']

    # close the file
    file.close()
    return wav

In [14]:
wav_list = []
for index, row in train_df.iterrows():
    if len(load_wav_pkl(row['file'])) == 160000:
        wav_list.append(load_wav_pkl(row['file']))
    if len(load_wav_pkl(row['file'])) != 160000: # if the waveform is shorter (for unknown reason)
        temp_wav = load_wav_pkl(row['file'])
        print('shorter length: '+str(len(temp_wav)))
        wav_list.append(np.pad(temp_wav, (0,160000-len(temp_wav)),'mean')) # zero-padding at the end to 160000
    


shorter length: 146099
shorter length: 159880
shorter length: 159997
shorter length: 159992
shorter length: 151683
shorter length: 148006
shorter length: 159880
shorter length: 156480
shorter length: 159999
shorter length: 159993
shorter length: 159880
shorter length: 157848
shorter length: 153357
shorter length: 151461


In [15]:
main_ds = tf.data.Dataset.from_tensor_slices((np.stack(wav_list, axis = 0), train_df['category']))
main_ds.element_spec

(TensorSpec(shape=(160000,), dtype=tf.float32, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

In [20]:
# applies the embedding extraction model to a wav data
def extract_embedding(wav_data, label, fold):
    # run YAMNet to extract embedding from the wav data
    scores, embeddings, spectrogram = yamnet_model(wav_data)
    num_embeddings = tf.shape(embeddings)[0]
    return (embeddings,
            tf.repeat(label, num_embeddings))

# extract embedding
main_ds = main_ds.map(extract_embedding).unbatch()
main_ds.element_spec

TypeError: in user code:


    TypeError: tf__extract_embedding() missing 1 required positional argument: 'fold'
